# NamasTea Project: Data Preprocessing and Analysis 

First, I import all the libraries I will need in the assignment


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.1               |             py_0          26 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

    altair:  4.1.0-py_1 conda-forge
    branca:  0.4.1-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


bran

## 1. Creation of the Toronto Dataframe 

In [2]:
# import the library we use to open URLs
import requests

# import the BeautifulSoup library 
!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup
!pip install lxml


# specify and open the url using urllib.request and put the HTML into the page variable
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

# parse the HTML from our URL 
soup = BeautifulSoup(req.content,'lxml')

table = soup.find_all('table')[0]

df = pd.read_html(str(table))

neighborhood=pd.DataFrame(df[0])

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.1       |   py36h9f0ad1d_0         163 KB  conda-forge

The following packages will be UPDATED:

    beautifulsoup4: 4.7.1-py36_1 --> 4.9.1-py36h9f0ad1d_0 conda-forge


beautifulsoup4-4.9.1 | 163 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [3]:
neighborhood.head()


,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
neighborhood.tail()


,Postal Code,Borough,Neighborhood
175,M5Z,Not assigned,NaN
176,M6Z,Not assigned,NaN
177,M7Z,Not assigned,NaN
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."
179,M9Z,Not assigned,NaN


**Only process the cells that have an assigned borough**


In [5]:
neighborhood=neighborhood[neighborhood.Borough != 'Not assigned']

neighborhood.tail()

,Postal Code,Borough,Neighborhood
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing Centre
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


**Combine more than one neighborhood when they exist in the same  postal code area using COMMA**


In [6]:
neighborhood['Neighborhood'].replace(r' \/ ', ', ', regex=True, inplace=True)

neighborhood.tail()

,Postal Code,Borough,Neighborhood
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing Centre
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


**Assign borough to neighborhood when neighborhood is Not assigned neighborhood**


In [7]:
neighborhood['Neighborhood']=neighborhood['Neighborhood'].replace('Not assigned', neighborhood['Borough'])
neighborhood.shape


(103, 3)

## 2. Get the latitude and the longitude coordinates of each neighborhood


**Create a dataframe of the coordinates**

In [8]:
!wget -q -O 'toronto_geospatial_data.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [9]:
torontogeo= pd.read_csv('toronto_geospatial_data.csv')
torontogeo


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


**Create a unique dataframe** 

In [10]:
toronto_df = neighborhood.merge(torontogeo)
toronto_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## 3. Explore the Neighborhoods


**Create map of Toronto using latitude and longitude values**


In [11]:
#geographical coordinates for Toronto

address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [12]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

**Define my Foursquare details**

In [13]:

CLIENT_ID = 'IB43C4M5C1VPP0XRNG2SSB2R1VQUWVYWODFN1114FWQVLR5Q' 
CLIENT_SECRET = 'N3NWM2XUOXRZDJOPYAY13XA5REUKXSRDEX1TCF23GD5VQJ0O' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: IB43C4M5C1VPP0XRNG2SSB2R1VQUWVYWODFN1114FWQVLR5Q
CLIENT_SECRET:N3NWM2XUOXRZDJOPYAY13XA5REUKXSRDEX1TCF23GD5VQJ0O


**Define a faunction to explore the neighborhoods venues**


In [14]:

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius


def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

**Create a dataframe with all neighborhoods' venues**

In [15]:

Toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],latitudes=toronto_df['Latitude'], longitudes=toronto_df['Longitude'])

print(Toronto_venues.shape)
Toronto_venues.head()

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview
The Danforth West, Ri

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Sun Life,43.754760,-79.332783,Construction & Landscaping
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [16]:
# check the venues returned for each neighborhood 

Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",21,21,21,21,21,21
Berczy Park,55,55,55,55,55,55
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
Business reply mail Processing Centre,18,18,18,18,18,18


## 4. Identify neighborhoods with higher education facilities 

**Search for Univerties --- **

In [50]:
search_query = ('college')
radius = 500
print(search_query + ' .... OK!')

college .... OK!


**Define the corresponding URL**

In [51]:
latitudes=toronto_df['Latitude']
longitudes=toronto_df['Longitude']

In [52]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=IB43C4M5C1VPP0XRNG2SSB2R1VQUWVYWODFN1114FWQVLR5Q&client_secret=N3NWM2XUOXRZDJOPYAY13XA5REUKXSRDEX1TCF23GD5VQJ0O&ll=43.6534817,-79.3839347&v=20180605&query=college&radius=500&limit=100'

**Send the GET Request and examine the results**

In [53]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ec400cafb34b5001b2db2ff'},
 'response': {'venues': [{'id': '4ad4c064f964a52065f820e3',
    'name': 'Ontario College of Art and Design University (OCADU)',
    'location': {'address': '100 McCaul St',
     'crossStreet': 'at Dundas St W',
     'lat': 43.65280251171013,
     'lng': -79.3910743699992,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.65280251171013,
       'lng': -79.3910743699992}],
     'distance': 579,
     'postalCode': 'M5T 1W1',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['100 McCaul St (at Dundas St W)',
      'Toronto ON M5T 1W1',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1ae941735',
      'name': 'University',
      'pluralName': 'Universities',
      'shortName': 'University',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/education/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId'

**Get relevant part of JSON and transform it into a dataframe**

In [54]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1ae941735', 'name': 'U...",False,4ad4c064f964a52065f820e3,100 McCaul St,CA,Toronto,Canada,at Dundas St W,579,"[100 McCaul St (at Dundas St W), Toronto ON M5...","[{'label': 'display', 'lat': 43.65280251171013...",43.652803,-79.391074,M5T 1W1,ON,Ontario College of Art and Design University (...,v-1589903718,NaN
1,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",False,4ff2e8b3e4b0a148f8d63b1b,180 Dundas Street West,CA,Toronto,Canada,NaN,317,"[180 Dundas Street West, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.655643, 'lng':...",43.655643,-79.386504,NaN,ON,College Of Respiratory Therapists Of Ontario,v-1589903718,NaN
2,"[{'id': '4d954b06a243a5684965b473', 'name': 'R...",False,4faac3d5e4b06c670e1afcc4,424 Yonge,CA,Toronto,Canada,College and Yonge,260,"[424 Yonge (College and Yonge), Toronto ON, Ca...","[{'label': 'display', 'lat': 43.65293604935704...",43.652936,-79.387083,NaN,ON,College Park Residences,v-1589903718,NaN
3,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",False,4ed5111929c2bbe66189dbde,438 University Ave.,CA,Toronto,Canada,University/Dundas,363,"[438 University Ave. (University/Dundas), Toro...","[{'label': 'display', 'lat': 43.65443940990297...",43.654439,-79.388243,NaN,ON,College Of Early Childhood Educators,v-1589903718,NaN
4,"[{'id': '4bf58dd8d48988d1ab941735', 'name': 'S...",False,5187c645498e659525992bef,150 Dundas Street West,CA,Toronto,Canada,NaN,246,"[150 Dundas Street West, Toronto ON M5G 1C6, C...","[{'label': 'display', 'lat': 43.65539371563695...",43.655394,-79.385483,M5G 1C6,ON,SGIC - Saint George International College,v-1589903718,NaN


**Define information of interest and filter dataframe**

In [55]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Ontario College of Art and Design University (...,University,100 McCaul St,CA,Toronto,Canada,at Dundas St W,579,"[100 McCaul St (at Dundas St W), Toronto ON M5...","[{'label': 'display', 'lat': 43.65280251171013...",43.652803,-79.391074,M5T 1W1,ON,4ad4c064f964a52065f820e3
1,College Of Respiratory Therapists Of Ontario,Office,180 Dundas Street West,CA,Toronto,Canada,NaN,317,"[180 Dundas Street West, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.655643, 'lng':...",43.655643,-79.386504,NaN,ON,4ff2e8b3e4b0a148f8d63b1b
2,College Park Residences,Residential Building (Apartment / Condo),424 Yonge,CA,Toronto,Canada,College and Yonge,260,"[424 Yonge (College and Yonge), Toronto ON, Ca...","[{'label': 'display', 'lat': 43.65293604935704...",43.652936,-79.387083,NaN,ON,4faac3d5e4b06c670e1afcc4
3,College Of Early Childhood Educators,Office,438 University Ave.,CA,Toronto,Canada,University/Dundas,363,"[438 University Ave. (University/Dundas), Toro...","[{'label': 'display', 'lat': 43.65443940990297...",43.654439,-79.388243,NaN,ON,4ed5111929c2bbe66189dbde
4,SGIC - Saint George International College,Student Center,150 Dundas Street West,CA,Toronto,Canada,NaN,246,"[150 Dundas Street West, Toronto ON M5G 1C6, C...","[{'label': 'display', 'lat': 43.65539371563695...",43.655394,-79.385483,M5G 1C6,ON,5187c645498e659525992bef
5,Herzing College,General College & University,220 Yonge St,CA,Toronto,Canada,NaN,247,"[220 Yonge St, Toronto ON m5b 2h1, Canada]","[{'label': 'display', 'lat': 43.65455268367057...",43.654553,-79.381242,m5b 2h1,ON,4cc5c3c3f49676b02a8273d5
6,College Optical,Eye Doctor,555 university ave,CA,Toronto,Canada,NaN,551,"[555 university ave, Toronto ON m5v 1x8, Canada]","[{'label': 'display', 'lat': 43.65739368160872...",43.657394,-79.388130,m5v 1x8,ON,4d9247bc639d14565f1b406c
7,Greystone College,College Technology Building,NaN,CA,NaN,Canada,NaN,341,[Canada],"[{'label': 'display', 'lat': 43.65479693006243...",43.654797,-79.387760,NaN,NaN,555c8e9e498ebde0730b3ca7
8,College of Physiotherapists of Ontario,Office,375 University Ave,CA,NaN,Canada,NaN,581,"[375 University Ave, Canada]","[{'label': 'display', 'lat': 43.6482638143444,...",43.648264,-79.384152,NaN,NaN,4fbe6809e4b09e81e6a8d571
9,The Royal College of Dentists of Canada,Office,"180 Dundas Street West, Suite 2003",CA,Toronto,Canada,NaN,330,"[180 Dundas Street West, Suite 2003, Toronto O...","[{'label': 'display', 'lat': 43.655805, 'lng':...",43.655805,-79.386498,M5G 1Z8,ON,4d6d0fc0cb7a8cfaf4665979


**Let's visualize the higher education vacilities in Toronto**

In [56]:
dataframe_filtered.name

0     Ontario College of Art and Design University (...
1          College Of Respiratory Therapists Of Ontario
2                               College Park Residences
3                  College Of Early Childhood Educators
4             SGIC - Saint George International College
5                                       Herzing College
6                                       College Optical
7                                     Greystone College
8                College of Physiotherapists of Ontario
9               The Royal College of Dentists of Canada
10                            Ontario College Of Trades
11                    Community College of Philadelphia
12                              21 college, CCIM office
13                                    Evergreen College
14                                         Fit for Life
15                                  Yonge-Dundas Square
Name: name, dtype: object

In [57]:
education_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

# add the higher educationion facilities as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(education_map)

# display map
education_map

## 5. Create and explore a dataframe with only the neighboorhoods with higher education facilities

In [ ]:
higher_education = toronto_df[toronto_df['Neighborhood'] == bla bla bla bla ].reset_index(drop=True)
higher_education

**Analyze each neighborhood**

In [47]:
# one hot encoding
Toronto_onehot = pd.get_dummies(toronto_df[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = toronto_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()
Toronto_onehot.shape

KeyError: "None of [Index(['Venue Category'], dtype='object')] are in the [columns]"

In [ ]:
#group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped
Toronto_grouped.shape

**5 most common venues for each neighborhood**

In [ ]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')